# Descarga y preparación del entorno

## Instalación de dependencias

In [ ]:
! pip install -r requirements.txt   

## Descomprimir datos
En el repositorio se incluye un archivo comprimido con los datasets en formato `.csv` de los créditos presupuestarios utilizados en el trabajo.

Estos fueron descargados el 5/10/2024 de la página de [presupuesto abierto](https://www.presupuestoabierto.gob.ar/sici/datos-abiertos) del gobierno nacional argentino. Los archivos utilizados se encuentran en la página como "Presupuesto de gastos y su ejecución detallada - agrupación anual".

También se incluye un script de python para descargar todos los datasets de este tipo disponibles al momento (desde 1995 hasta 2024). Se puede encontrar en `utils/descarga-presupuestos.py`.

In [2]:
import zipfile
import os

# Cambiar al directorio "credito-anual"
os.chdir('credito-anual')

# Descomprimir el archivo "credito-anual.zip"
with zipfile.ZipFile('credito-anual.zip', 'r') as zip_ref:
    zip_ref.extractall()
    
os.chdir('..')

print('Descomprimido credito-anual.zip')

Descomprimido credito-anual.zip


# Manejo de los datos con pandas

In [1]:
# Cargamos las bases a varios dataframes
import pandas as pd
DFs = {}
for year in range(2012,2020):
  df = pd.read_csv(f'credito-anual/credito-anual-{year}.csv', decimal=',')
  DFs[year] = df
  
# Generar DataFrames para los periodos 2012-2015, 2016-2019 y uno para un periodo total 2012-2019
df_2012_2015 = pd.concat([DFs[year] for year in range(2012, 2016)], ignore_index=True)
df_2016_2019 = pd.concat([DFs[year] for year in range(2016, 2020)], ignore_index=True)
df_total = pd.concat(DFs.values(), ignore_index=True)

In [6]:
# Filtramos los registros que contienen la palabra "Trabajo" en la columna "funcion_desc"
df_2012_2015_trabajo = df_2012_2015[df_2012_2015['funcion_desc'].str.contains('Trabajo', na=False)]
df_2016_2019_trabajo = df_2016_2019[df_2016_2019['funcion_desc'].str.contains('Trabajo', na=False)]
df_total_trabajo = df_total[df_total['funcion_desc'].str.contains('Trabajo', na=False)]

## Calculo de inversión con funcion 'Trabajo' por períodos estudiados

In [15]:
trabajo_2012_2015= df_2012_2015_trabajo['credito_pagado'].sum()
trabajo_2016_2019 = df_2016_2019_trabajo['credito_pagado'].sum()
trabajo_total = df_total_trabajo['credito_pagado'].sum()

total_2012_2015 = df_2012_2015['credito_pagado'].sum()
total_2016_2019 = df_2016_2019['credito_pagado'].sum()
total_total = df_total['credito_pagado'].sum()

print('2012-2015:')
print(f'    Total pagado con función trabajo: {trabajo_2012_2015:,.2f}\n',
        f'   Total de pagos: {total_2012_2015:,.2f}')
print(f'    Corresponde al {trabajo_2012_2015/total_2012_2015:.2%} del total')

print('2016-2019:')
print(f'    El total pagado con función trabajo: {trabajo_2016_2019:,.2f}\n',
        f'   El total de pagos: {total_2016_2019:,.2f}')
print(f'    Corresponde al {trabajo_2016_2019/total_2016_2019:.2%} del total')

print('2012-2019:')
print(f'   El total pagado con función trabajo: {trabajo_total:,.2f} \n',
        f'El total de pagos fue de {total_total:,.2f}')
print(f'    Corresponde al {trabajo_total/total_total:.2%} del total')
print('\n(En miles de millones de pesos)')

2012-2015:
    Total pagado con función trabajo: 17,070.08
    Total de pagos: 3,673,561.01
    Corresponde al 0.46% del total
2016-2019:
    El total pagado con función trabajo: 33,300.60
    El total de pagos: 12,317,476.66
    Corresponde al 0.27% del total
2012-2019:
   El total pagado con función trabajo: 50,370.68 
 El total de pagos fue de 15,991,037.68
    Corresponde al 0.31% del total

(En miles de millones de pesos)


## Calculo de inversión con funcion 'Trabajo' por año

In [14]:
porcentaje_por_año = {}
print("Porcentaje de función \'Trabajo\' en todos los créditos pagados:")
for year in range(2012,2020):
    total_pagado = DFs[year]['credito_pagado'].sum()
    total_trabajo = DFs[year][DFs[year]['funcion_desc'].str.contains('Trabajo', na=False)]['credito_pagado'].sum()
    porcentaje_por_año[year] = total_trabajo/total_pagado
    print(f"{year}: {porcentaje_por_año[year]:.2%}") 

Porcentaje de función 'Trabajo' en todos los créditos pagados:
2012: 0.58%
2013: 0.50%
2014: 0.39%
2015: 0.46%
2016: 0.31%
2017: 0.39%
2018: 0.23%
2019: 0.22%
